In [ ]:
# MIT License

# Copyright (c) 2025 Mehran Sharifi

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# Combined natural convection solver in L- and C-shaped enclosures using FEniCS
# Modified and extended by Mehran Sharifi, 2025.
# -----------------------------------------------------------------------------

from fenics import *
from mshr import *
import matplotlib.pyplot as plt

# ----- CONFIGURATION -----
geometry_type = 'L'  # Options: 'L' or 'C'
mesh_resolution = 64
Pr_val = 0.71
Ra_val = 1e6

# ----- DOMAIN CREATION -----
if geometry_type == 'L':
    domain = Rectangle(Point(0, 0), Point(1, 1)) - Rectangle(Point(0.9, 0.5), Point(1.0, 1.0))
else:  # C-shaped
    outer = Rectangle(Point(0, 0), Point(1, 1))
    cutout = Rectangle(Point(0.9, 0.35), Point(1.0, 0.65))
    domain = outer - cutout

mesh = generate_mesh(domain, mesh_resolution)

# ----- FUNCTION SPACE -----
P1 = FiniteElement('P', mesh.ufl_cell(), 1)
P2 = VectorElement('P', mesh.ufl_cell(), 2)
mixed_element = MixedElement([P1, P2, P1])
W = FunctionSpace(mesh, mixed_element)

# ----- BOUNDARIES -----
class HotWall(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and near(x[0], 0.0)

class ColdWall(SubDomain):
    def inside(self, x, on_boundary):
        if geometry_type == 'L':
            return on_boundary and near(x[0], 1.0) and x[1] <= 0.5
        else:
            return (on_boundary and (
                (near(x[0], 1.0) and (x[1] <= 0.35 + DOLFIN_EPS or x[1] >= 0.65 - DOLFIN_EPS)) or
                (near(x[0], 0.9) and (0.35 - DOLFIN_EPS <= x[1] <= 0.65 + DOLFIN_EPS))
            ))

class ColdWall2(SubDomain):
    def inside(self, x, on_boundary):
        return geometry_type == 'L' and on_boundary and near(x[0], 0.9) and x[1] >= 0.5

# ----- APPLY BOUNDARIES -----
hot_wall = HotWall()
cold_wall = ColdWall()
cold_wall2 = ColdWall2() if geometry_type == 'L' else None

# ----- CONSTANTS -----
T_h = Constant(1.0)
T_c = Constant(0.0)
mu = Constant(1.0)
Pr = Constant(Pr_val)
Ra = Constant(Ra_val)

# ----- VARIABLES -----
psi_p, psi_u, psi_T = TestFunctions(W)
w = Function(W)
p, u, T = split(w)

# Initial condition
w_n = interpolate(
    Expression(("0.", "0.", "0.", "T_h + x[0]*(T_c - T_h)"), T_h=1.0, T_c=0.0, element=mixed_element),
    W
)
p_n, u_n, T_n = split(w_n)

# Time stepping
dt_val = 0.001
dt = Constant(dt_val)
u_t = (u - u_n)/dt
T_t = (T - T_n)/dt

# ----- WEAK FORMULATION -----
f_B = Ra / Pr * T * Constant((0., -1.))
inner, dot, grad, div, sym = inner, dot, grad, div, sym

mass = -psi_p*div(u)
momentum = dot(psi_u, u_t + dot(grad(u), u) + f_B) - div(psi_u)*p + 2.*mu*inner(sym(grad(psi_u)), sym(grad(u)))
energy = psi_T*T_t + dot(grad(psi_T), 1./Pr*grad(T) - T*u)
F = (mass + momentum + energy)*dx

# Pressure stabilization
gamma = Constant(1e-7)
F += -psi_p*gamma*p*dx
JF = derivative(F, w, TrialFunction(W))

# ----- BOUNDARY CONDITIONS -----
bcs = [
    DirichletBC(W.sub(1), Constant((0.0, 0.0)), 'on_boundary'),
    DirichletBC(W.sub(2), T_h, hot_wall),
    DirichletBC(W.sub(2), T_c, cold_wall)
]
if geometry_type == 'L':
    bcs.append(DirichletBC(W.sub(2), T_c, cold_wall2))

# ----- SOLVER -----
problem = NonlinearVariationalProblem(F, w, bcs, JF)
M = inner(grad(T), grad(T))*dx
epsilon_M = 0.05
solver = AdaptiveNonlinearVariationalSolver(problem, M)
w.leaf_node().vector()[:] = w_n.leaf_node().vector()
solver.solve(epsilon_M)

# ----- PLOTTING -----
def plot_solution(w, step):
    p, u, T = split(w.leaf_node())
    plot(T)
    plt.title(f"Temperature Field (Step {step})")
    plt.xlabel('x')
    plt.ylabel('y')
    plt.colorbar()
    plt.show()

# ----- TIME LOOP -----
for step in range(15):
    print(f"Step {step+1}, dt = {dt_val}")
    w_n.leaf_node().vector()[:] = w.leaf_node().vector()
    dt_val *= 2.
    dt.assign(dt_val)
    solver.solve(epsilon_M)
    plot_solution(w, step+1)

Generating forms required for error control, this may take some time...
